## Search space autogeneration

This notebook describes how to generate search space automaticaly from **EfficientNet-V2 Small** model.

### Main chapters of this notebook:
1. Setup the environment
1. Prepare dataset and create dataloaders
1. Generate model with search variants and move it into search space
1. Check pretrain, search and tune phases

## Setup the environment
First, let's set up the environment and make some common imports.

In [ ]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# You may need to change this variable to match free GPU index
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from pathlib import Path

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_optimizer import RAdam

from enot.autogeneration.search_variants_from_model import generate_pruned_search_variants_model
from enot.latency import min_latency
from enot.latency import max_latency
from enot.latency import best_arch_latency
from enot.models import SearchSpaceModel
from enot.optimize import EnotPretrainOptimizer
from enot.optimize import EnotSearchOptimizer

from tutorial_utils.train import accuracy
from tutorial_utils.train import WarmupScheduler

from tutorial_utils.checkpoints import download_autogen_pretrain_checkpoint
from tutorial_utils.checkpoints import download_autogen_search_checkpoint
from tutorial_utils.dataset import create_imagenette_dataloaders
from tutorial_utils.phases import tutorial_pretrain_loop
from tutorial_utils.phases import tutorial_search_loop
from tutorial_utils.phases import tutorial_train_loop

from timm.models.factory import create_model

### In the following cell we setup all necessary dirs

* `ENOT_HOME_DIR` - ENOT framework home directory
* `ENOT_DATASETS_DIR` - root directory for datasets (imagenette2, ...)
* `PROJECT_DIR` - project directory to save training logs, checkpoints, ...

In [ ]:
ENOT_HOME_DIR = Path.home() / '.enot'
ENOT_DATASETS_DIR = ENOT_HOME_DIR / 'datasets'
PROJECT_DIR = ENOT_HOME_DIR / 'search_space_autogeneration'

ENOT_HOME_DIR.mkdir(exist_ok=True)
ENOT_DATASETS_DIR.mkdir(exist_ok=True)
PROJECT_DIR.mkdir(exist_ok=True)

## Prepare dataset and create dataloaders

In [ ]:
dataloaders = create_imagenette_dataloaders(
    dataset_root_dir=ENOT_DATASETS_DIR, 
    project_dir=PROJECT_DIR,
    input_size=(224, 224),
    batch_size=32,
)

## Generate model with search variants and move it into search space

**IMPORTANT:**<br>
<span style="color:red">**Weights of source model are used for initialization of search variants parameters, so pretrained models are necessary for good results.**</span>.

In some cases it is necessary to specify modules that should remain unchanged while generating model.

There are two options:
1. Specify particular module.
1. Specify module type: in this case all modules with specified type remain unchanged.

To do this, pass particular modules or types of modules that need to remain unchaged as parameter in `generate_pruned_search_variant_model`.

In [ ]:
my_model = create_model('efficientnetv2_s', pretrained=False)
classifier = my_model.get_classifier()
my_model.classifier = nn.Linear(
    in_features=classifier.in_features, 
    out_features=10, 
    bias=True,
)
my_model.eval()

some_excluded_block = my_model.blocks[4][1]
generated_model = generate_pruned_search_variants_model(
    my_model, 
    width_multipliers=(0.0, 0.25, 0.5, 0.75, 1.0),
    excluded_modules=[some_excluded_block], # Leave some_excluded_block unchanged.
)
# move model to search space
search_space = SearchSpaceModel(generated_model).cuda()

## Check pretrain, search and tune phases

In this tutorial we use the same pretrain/search/train loops as in "Tutorial - getting started".

**IMPORTANT:**<br>
We set `N_EPOCHS`= 3 in this example to make tutorial execution faster. This is not enough for good pretrain quality, and you should set `N_EPOCHS`>= 100 if you want to achieve good results.

In [ ]:
N_EPOCHS = 3
N_WARMUP_EPOCHS = 1
len_train = len(dataloaders['pretrain_train_dataloader'])

optimizer = SGD(params=search_space.model_parameters(), lr=0.06, momentum=0.9, weight_decay=1e-4)
enot_optimizer = EnotPretrainOptimizer(search_space=search_space, optimizer=optimizer)
scheduler = CosineAnnealingLR(optimizer, T_max=len_train*N_EPOCHS, eta_min=1e-8)
scheduler = WarmupScheduler(scheduler, warmup_steps=len_train*N_WARMUP_EPOCHS)
loss_function = nn.CrossEntropyLoss().cuda()

tutorial_pretrain_loop(
    epochs=N_EPOCHS,
    search_space=search_space,
    enot_optimizer=enot_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['pretrain_train_dataloader'],
    validation_loader=dataloaders['pretrain_validation_dataloader'],
    scheduler=scheduler,
)

In [ ]:
optimizer = RAdam(search_space.architecture_parameters(), lr=0.01)
enot_optimizer = EnotSearchOptimizer(search_space=search_space, optimizer=optimizer)

tutorial_search_loop(
    epochs=5,
    search_space=search_space,
    enot_optimizer=enot_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['search_train_dataloader'],
    validation_loader=dataloaders['search_validation_dataloader'],
    latency_loss_weight=1e-3,
    latency_type='mmac.thop',
    scheduler=None,
)

In [ ]:
# get regular model with the best architecture
best_model = search_space.get_network_with_best_arch().cuda()

In [ ]:
optimizer = RAdam(best_model.parameters(), lr=1e-3, weight_decay=1e-4)

tutorial_train_loop(
    epochs=5,
    model=best_model,
    optimizer=optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['tune_train_dataloader'],
    validation_loader=dataloaders['tune_validation_dataloader'],
    scheduler=None,
)